In [2]:
import sys, os, pickle
sys.path.append('/scratch/oem214/vanilla-rtrl/')
from cluster import *
from continual_learning import *
from core import *
from dynamics import *
from functions import *
from gen_data import *
from learning_algorithms import *
from optimizers import *
from plotting import *
from wrappers import *

In [3]:
### --- SET UP ALL CONFIGS --- ###
from itertools import product
n_seeds = 20
macro_configs = config_generator(mu=[0, 0.3, 0.6, 0.9],
                                 clip=[0.1, 0.3, 1, 3, 10, 1000])
micro_configs = tuple(product(macro_configs, list(range(n_seeds))))

### --- SELECT PARTICULAR CONFIG --- ###
try:
    i_job = int(os.environ['SLURM_ARRAY_TASK_ID']) - 1
except KeyError:
    i_job = 0
params, i_seed = micro_configs[i_job]
i_config = i_job//n_seeds

new_random_seed_per_condition = True
if new_random_seed_per_condition:
    np.random.seed(i_job)
else: #Match random seeds across conditions
    np.random.seed(i_seed)

In [ ]:
task = Flip_Flop_Task(3, 0.05, input_magnitudes=None)
N_train = 1000000
N_test = 5000
checkpoint_interval = None
sigma = 0

data = task.gen_data(N_train, N_test)

n_in = task.n_in
n_hidden = 32
n_out = task.n_out
W_in  = np.random.normal(0, np.sqrt(1/(n_in)), (n_hidden, n_in))
W_rec = np.random.normal(0, np.sqrt(1/n_hidden), (n_hidden, n_hidden))
W_out = np.random.normal(0, np.sqrt(1/(n_hidden)), (n_out, n_hidden))

b_rec = np.zeros(n_hidden)
b_out = np.zeros(n_out)

alpha = 1
sigma = 0

rnn = RNN(W_in, W_rec, W_out, b_rec, b_out,
          activation=tanh,
          alpha=alpha,
          output=identity,
          loss=mean_squared_error)

learn_alg = RFLO(rnn, alpha=alpha, L2_reg=0.0001, L1_reg=0.0001, )
optimizer = SGD_Momentum(lr=0.0001, mu=params['mu'], clip_norm=params['clip'])

monitors = []

sim = Simulation(rnn)
sim.run(data, learn_alg=learn_alg, optimizer=optimizer,
        sigma=sigma,
        monitors=monitors,
        verbose=True,
        report_accuracy=False,
        report_loss=True,
        checkpoint_interval=checkpoint_interval)

In [ ]:
test_sim = Simulation(rnn)
test_sim.run(data, mode='test', monitors=['rnn.loss_'], verbose=False)
processed_data = np.mean(test_sim.mons['rnn.loss_'])

In [ ]:
### --- SAVE RESULTS -- ###
result = {'sim': sim, 'i_seed': i_seed, 'task': task,
          'config': params, 'i_config': i_config, 'i_job': i_job}
try:
    result['processed_data'] = processed_data
except NameError:
    pass
    
try:
    save_dir = os.environ['SAVEDIR']
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    save_path = os.path.join(save_dir, 'result_{}'.format(i_job))

    with open(save_path, 'wb') as f:
        pickle.dump(result, f)
except KeyError:
    pass

In [4]:
###Truncate file above
file_name = 'pipeline_test'
job_name = 'rtrl_low_lr'
get_ipython().run_cell_magic('javascript', '', 'IPython.notebook.save_notebook()')
get_ipython().system('jupyter nbconvert --to script --no-prompt {}.ipynb'.format(file_name))
get_ipython().system('awk "/###Truncate/ {{exit}} {{print}}" {}.py'.format(file_name))
get_ipython().system('sed -i "/###Truncate/Q" {}.py'.format(file_name))
get_ipython().system('mv {}.py ../cluster_main_scripts/{}.py'.format(file_name, job_name))

<IPython.core.display.Javascript object>

[NbConvertApp] Converting notebook pipeline_test.ipynb to script
[NbConvertApp] Writing 4610 bytes to pipeline_test.py
awk: 1: unexpected character '.'


In [5]:
###Submit job to cluster
n_jobs = len(micro_configs)
write_job_file(job_name, py_file_name='{}.py'.format(job_name))
job_id_1 = submit_job('../job_scripts/{}.s'.format(job_name), n_jobs)

sending incremental file list
vanilla-rtrl/.idea/
vanilla-rtrl/.idea/workspace.xml
vanilla-rtrl/cluster/
vanilla-rtrl/cluster/submit_jobs.py
vanilla-rtrl/cluster/__pycache__/
vanilla-rtrl/cluster/__pycache__/submit_jobs.cpython-37.pyc
vanilla-rtrl/cluster/__pycache__/submit_jobs.cpython-38.pyc

sent 68,062 bytes  received 134 bytes  136,392.00 bytes/sec
total size is 1,244,658  speedup is 18.25


In [ ]:
job_name = 'rtrl_low_lr'

In [7]:
###Get job status
get_ipython().system('squeue -u oem214')

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
       7876658_476        cm rtrl_low   oem214  R       0:31      1 cm22
       7876658_477        cm rtrl_low   oem214  R       0:31      1 cm22
       7876658_457        cm rtrl_low   oem214  R       0:31      1 cm15
       7876658_475        cm rtrl_low   oem214  R       0:31      1 cm10
       7876658_478        cm rtrl_low   oem214  R       0:31      1 cm32
       7876658_479        cm rtrl_low   oem214  R       0:31      1 cm28
       7876658_444        cm rtrl_low   oem214  R       0:34      1 cm23
       7876658_446        cm rtrl_low   oem214  R       0:34      1 cm23
       7876658_437        cm rtrl_low   oem214  R       0:35      1 cm23
       7876658_439        cm rtrl_low   oem214  R       0:35      1 cm23
       7876658_429        cm rtrl_low   oem214  R       0:36      1 cm20
       7876658_425        cm rtrl_low   oem214  R       0:37      1 cm20
       7876658_416        cm rtrl_low  

In [ ]:
configs_array, results_array, key_order, sim_dict = unpack_processed_data('../job_scripts/{}.s'.format(job_name))

In [ ]:
def plot_2d_array_of_config_results(configs_array, results_array, key_order,
                                    log_scale=False):
    
    fig = plt.figure()

    plt.imshow(results_array.mean(-1))
    
    if log_scale:
        plt.yticks(range(results_array.shape[0]), np.round(np.log10(configs_array[key_order[0]]), 3))
        plt.xticks(range(results_array.shape[1]), np.round(np.log10(configs_array[key_order[1]]), 3))
    else:
        plt.yticks(range(results_array.shape[0]), np.round(configs_array[key_order[0]], 3))
        plt.xticks(range(results_array.shape[1]), np.round(configs_array[key_order[1]], 3))
        
    
    plt.ylabel(key_order[0])
    plt.xlabel(key_order[1])
    plt.colorbar()

In [ ]:
plot_2d_array_of_config_results(configs_array, results_array, key_order, log_scale=True)